In [1]:
import numpy as np 
import pandas as pd 
import os 
import h5py
from tqdm import tqdm
import pyarrow
from datasets import load_dataset 
import torch 
import torch.nn as nn 
import time 
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt 
import torch.nn.functional as F
import sys 
sys.path.append('../code')
import utils
import model 
import loss 

d:\ANACONDA\ENVS\SCVI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## allen brain 小鼠海马 21年工作

f = h5py.File(r"D:\sly_data\mus_brain_ref_data\CTX_Hip_counts_10x.h5",'r')
#anno = pd.read_csv(r"D:\sly_data\mus_brain_ref_data\CTX_Hip_anno_10x.csv\CTX_Hip_anno_10x\CTX_Hip_anno_10x.csv")

## 这里f['data']['count']的数值类型是Int32

gene = f['data']['gene'][:].astype('U')
sample = f['data']['samples'][:].astype('U')
#print(anno.shape, sample.shape)

In [3]:
gene2id_dic = {}
for i, _ in enumerate(gene):
    gene2id_dic[_] = i + 3 

gene2id_dic['<cls>'] = 0
#gene2id_dic['<mask>'] = 1
gene2id_dic['<pad>'] = 1
tokenizer = utils.gene_tokenizer(gene2id_dic)

In [4]:
part = load_dataset(path ="C:/Users/sunrui/Desktop/pretrain_project/notebook/allen_2021_data/")
train = part['train']
train

Dataset({
    features: ['cell_id', 'pos_gene_ids', 'pos_gene_exp', 'neg_gene_ids', 'pos_gene_num', 'umi_counts', 'meta_info'],
    num_rows: 200
})

In [5]:
preprocess_part = utils.preprocess_module(
    tokenizer,
    100, gex_cls_id = 101, gex_mask_id = 102, gex_pad_id = 103, max_num = 64, mask_ratio = 0.3, sample_rho = 0.9
)

In [6]:
train

Dataset({
    features: ['cell_id', 'pos_gene_ids', 'pos_gene_exp', 'neg_gene_ids', 'pos_gene_num', 'umi_counts', 'meta_info'],
    num_rows: 200
})

In [7]:
#dataset = train.map(preprocess_part.double_preprocess, batched = False, remove_columns= ['umi_counts', 'pos_gene_num', 'meta_info'])

In [60]:
#dataset = train.map(preprocess_part.preprocess, batched = False, remove_columns= ['umi_counts', 'pos_gene_num', 'meta_info'])
dataset = train.map(preprocess_part.double_preprocess, batched = True, remove_columns= ['umi_counts', 'pos_gene_num', 'meta_info'])
#print(dataset[0])

# 移除不需要的列
columns_to_remove = ['cell_id', 'pos_gene_ids', 'pos_gene_exp', 'neg_gene_ids']  # 指定不需要的列名
dataset = dataset.remove_columns(columns_to_remove)

# 查看处理后的数据
print(dataset[0])

dataset.set_format(type = 'torch')
print(dataset.format['type'])

#dataset = dataset.set_format(type='torch')
# 查看处理后的数据
#print(dataset[0])

'''
dataset = dataset.set_format(type='torch', columns = ['sampled_gene_ids',
                                                    'sampled_gene_exp',
                                                    'mask_array',
                                                    'pred_array'])
dataset
'''

Map:   0%|          | 0/200 [00:05<?, ? examples/s]


TypeError: unhashable type: 'list'

In [9]:
''' 
new_dataset = dataset.set_format(type='torch', columns = ['sampled_gene_ids',
                                                    'sampled_gene_exp',
                                                    'mask_array',
                                                    'pred_array'])
new_dataset
'''

" \nnew_dataset = dataset.set_format(type='torch', columns = ['sampled_gene_ids',\n                                                    'sampled_gene_exp',\n                                                    'mask_array',\n                                                    'pred_array'])\nnew_dataset\n"

In [10]:
# 定义 DataLoader 参数
batch_size = 32
shuffle = True
num_workers = 4

# 创建 DataLoader
data_loader = DataLoader(
    dataset, 
    batch_size=batch_size,
    shuffle=shuffle,
    num_workers=num_workers
)

# 迭代数据
for batch_idx, batch in enumerate(data_loader):
    sampled_gene_ids = batch['cell_0']
    print(f"Batch {batch_idx+1}:")
    #print("Features shape:", sampled_gene_ids.shape)

Batch 1:
Batch 2:
Batch 3:
Batch 4:
Batch 5:
Batch 6:
Batch 7:


In [11]:
count_embedding_num = 104
gene_embedding_num = tokenizer.vocab_size
d_model = 16
gene_padding_idx = 1
count_padding_idx = 103
n_head = 8
dim_ffn = 4*d_model
dropout = 0.1
layer_norm_eps =1e-5
batch_first = True
norm_first = False
num_layers = 4
norm = None
num_hiddens = 16

my_model = model.sc_pretrain(count_embedding_num,
                 gene_embedding_num,
                 d_model,
                 gene_padding_idx,
                 count_padding_idx,
                 n_head,
                 dim_ffn,
                 dropout,
                 layer_norm_eps,
                 batch_first,
                 norm_first,
                 num_layers,
                 norm,
                 num_hiddens)

In [12]:
sampled_gene_ids = torch.cat((batch['cell_0']['sampled_gene_ids'], batch['cell_1']['sampled_gene_ids']))
masked_gene_exp = torch.cat((batch['cell_0']['masked_gene_exp'], batch['cell_1']['masked_gene_exp']))
sampled_gene_exp = torch.cat((batch['cell_0']['sampled_gene_exp'], batch['cell_1']['sampled_gene_exp']))
mask_array = torch.cat((batch['cell_0']['mask_array'], batch['cell_1']['mask_array']))
pred_array = torch.cat((batch['cell_0']['pred_array'], batch['cell_1']['pred_array']))

my_model(sampled_gene_ids, sampled_gene_exp, mask_array, pred_array)


(tensor([[-5.6755e-01, -1.1789e+00, -3.0600e-03,  5.4675e-01,  7.0871e-03,
          -1.2441e+00,  8.7547e-01, -3.3778e-01, -6.8149e-02, -7.7570e-01,
          -2.4774e-02,  1.1267e-01, -2.3681e-01,  6.3370e-01,  5.5948e-02,
           2.2701e-01],
         [-7.8207e-01, -7.7466e-01, -6.9818e-01,  8.9053e-01,  2.7649e-01,
          -1.0635e+00,  1.6042e+00, -8.5112e-02, -4.1971e-01, -6.8761e-01,
           3.4051e-01,  5.8115e-05,  5.4345e-01,  5.7503e-01, -3.0667e-02,
          -7.3806e-02],
         [-4.5418e-01, -6.9307e-01, -6.6873e-02,  4.0483e-01,  7.1620e-02,
          -1.2318e+00,  1.0475e+00, -3.9450e-01, -1.0036e-01, -9.7177e-01,
           1.4319e-01,  4.4987e-01, -2.1180e-01,  7.3756e-01,  5.9642e-02,
           1.7191e-01],
         [-7.5992e-01, -7.8207e-01, -7.8818e-01,  7.0996e-01,  5.9590e-01,
          -1.2512e+00,  1.6066e+00, -2.7297e-01, -3.4565e-01, -7.7235e-01,
           2.7481e-01,  1.1409e-01,  4.3541e-01,  8.2427e-01, -6.6174e-03,
           3.7614e-01],
    

In [13]:
cell_embed, gene_pred = my_model(sampled_gene_ids, sampled_gene_exp, mask_array, pred_array)

In [14]:
import loss 

pretrain_loss = loss.pretrain_loss()

In [15]:
cell_embed.size(0)

16

In [16]:
pred_label = sampled_gene_exp[mask_array]
print(pred_label.shape)
print(np.unique(pred_label))

torch.Size([329])
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  18
  19  20  21  22  23  25  27  29  31  32  35  36  41  43  45  48  57  62
  64  66  69  80  87 100]


In [17]:
pretrain_loss(cell_embed[:8,:],cell_embed[8:,:], gene_pred, pred_label)

{'total_loss': tensor(7.1916, grad_fn=<AddBackward0>),
 'gene_exp_ce_loss': tensor(5.0024, grad_fn=<NllLossBackward0>),
 'clip_loss': tensor(2.1892, grad_fn=<DivBackward0>)}

In [18]:
torch.cuda.is_available()

False

In [30]:
import torch.optim as optim
my_model = train_one_epoch(my_model, 
                data_loader,
                pretrain_loss, 
                optimizer = optim.SGD(my_model.parameters(), lr=1e-3, momentum=0.9),
                device = 'cpu',
                gradient_accumulation_steps = 2,
                save_steps = 3,
                save_dir = 'test')

over_1
8.650686264038086
Finished Training


In [36]:
device = 'cpu'

# 训练模型
my_model.to(device)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# 训练模型
total_steps = 0  # 记录总的训练步数
accumulated_loss = 0.0  # 累积损失
accumulated_samples = 0  # 累积样本数
gradient_accumulation_steps = 2
save_steps = 3 
save_dir = 'test'


In [47]:

    

    # 计算累积损失和样本数
    accumulated_loss += loss.item() * cell_embed.size(0)
    accumulated_samples += cell_embed.size(0)

    # 反向传播
    loss.backward()

        # 梯度累积
    if (i + 1) % gradient_accumulation_steps == 0:
        print(i)
        optimizer.step()
        optimizer.zero_grad()  # 清空梯度
        total_steps += 1

        # 打印累积损失
        avg_loss = accumulated_loss / accumulated_samples
        print(f'Step {total_steps}, Loss: {avg_loss:.4f}')

        # 重置累积损失和样本数
        accumulated_loss = 0.0
        accumulated_samples = 0

        # 模型保存
        if total_steps % save_steps == 0:
            checkpoint = {
                'steps': total_steps,
                'model_state_dict': my_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss
            }
            torch.save(checkpoint, os.path.join(save_dir, f'checkpoint_epoch_{total_steps}.pth'))
            print(f"Checkpoint saved at steps {total_steps}")


9 8.342761993408203
9 8.397958755493164
1
Step 10, Loss: 8.2233
10 8.406044960021973
10 8.40825080871582
3
Step 11, Loss: 8.4071
11 8.223245620727539
11 8.413407325744629
5
Step 12, Loss: 8.3183
Checkpoint saved at steps 12
12 6.790136337280273


In [42]:
model

<module 'model' from 'c:\\Users\\sunrui\\Desktop\\pretrain_project\\notebook\\../code\\model.py'>

In [41]:
model.state_dict()

AttributeError: module 'model' has no attribute 'state_dict'

In [59]:
def train_one_epoch(model,trainloader,criterion,optimizer,device,gradient_accumulation_steps, save_steps, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # 定义损失函数和优化器
    #criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    # 训练模型
    model.to(device)
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    # 训练模型
    total_steps = 0  # 记录总的训练步数
    accumulated_loss = 0.0  # 累积损失
    accumulated_samples = 0  # 累积样本数

    for i, batch in enumerate(trainloader, 0):
        sampled_gene_ids = torch.cat((batch['cell_0']['sampled_gene_ids'], batch['cell_1']['sampled_gene_ids']))
        masked_gene_exp = torch.cat((batch['cell_0']['masked_gene_exp'], batch['cell_1']['masked_gene_exp']))
        sampled_gene_exp = torch.cat((batch['cell_0']['sampled_gene_exp'], batch['cell_1']['sampled_gene_exp']))
        mask_array = torch.cat((batch['cell_0']['mask_array'], batch['cell_1']['mask_array']))
        pred_array = torch.cat((batch['cell_0']['pred_array'], batch['cell_1']['pred_array'])) 

        sampled_gene_ids.to(device)
        masked_gene_exp.to(device)
        sampled_gene_exp.to(device)
        mask_array.to(device)
        pred_array.to(device)
        pred_label = sampled_gene_exp[mask_array]

        # 前向传播
        cell_embed, gene_pred = model(sampled_gene_ids, sampled_gene_exp, mask_array, pred_array)
        N = int(cell_embed.shape[0] / 2)
        cell_0 = cell_embed[:N,:]
        cell_1 = cell_embed[N:,:]
        loss = criterion(cell_0,cell_1, gene_pred, pred_label)
        loss = loss['total_loss']
        

        # 计算累积损失和样本数
        accumulated_loss += loss.item() * cell_embed.size(0)
        accumulated_samples += cell_embed.size(0)

        # 反向传播
        loss.backward()

        # 梯度累积
        if (i + 1) % gradient_accumulation_steps == 0:
            print(i)
            optimizer.step()
            optimizer.zero_grad()  # 清空梯度
            total_steps += 1

            # 打印累积损失
            avg_loss = accumulated_loss / accumulated_samples
            print(f'Step {total_steps}, Loss: {avg_loss:.4f}')

            # 重置累积损失和样本数
            accumulated_loss = 0.0
            accumulated_samples = 0

            # 模型保存
            if total_steps % save_steps == 0:
                checkpoint = {
                    'steps': total_steps,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': avg_loss
                }
                torch.save(checkpoint, os.path.join(save_dir, f'checkpoint_epoch_{total_steps}.pth'))
                print(f"Checkpoint saved at steps {total_steps}")

        if (i + 1) == len(trainloader):
            optimizer.step()
            optimizer.zero_grad()  # 清空梯度
            total_steps += 1

            # 打印累积损失
            avg_loss = accumulated_loss / accumulated_samples
            print(f'Step {total_steps}, Loss: {avg_loss:.4f}')

            # 重置累积损失和样本数
            accumulated_loss = 0.0
            accumulated_samples = 0

            # 模型保存
            if total_steps % save_steps == 0:
                checkpoint = {
                    'steps': total_steps,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': avg_loss
                }
                torch.save(checkpoint, os.path.join(save_dir, f'checkpoint_epoch_{total_steps}.pth'))
                print(f"Checkpoint saved at steps {total_steps}")


            print('Finished Training')

    return model
    
def train_multi_epoch(model,trainloader,criterion,optimizer,device,gradient_accumulation_steps, save_steps, save_dir, epochs):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i in range(epochs):
        print(f'epochs: {i} begin:' + '===='*20)
        model = train_one_epoch(model,trainloader,criterion,optimizer,device,gradient_accumulation_steps, save_steps, save_dir = os.path.join(save_dir,f'epoch_{i}'))
    return model


In [57]:
my_model = train_one_epoch(my_model, 
                data_loader,
                pretrain_loss, 
                optimizer = optim.SGD(my_model.parameters(), lr=1e-3, momentum=0.9),
                device = 'cpu',
                gradient_accumulation_steps = 2,
                save_steps = 3,
                save_dir = 'test')

1
Step 1, Loss: 8.2409
3
Step 2, Loss: 8.1907
5
Step 3, Loss: 8.2319
Checkpoint saved at steps 3
Step 4, Loss: 6.7744
Finished Training


In [58]:
my_model = train_multi_epoch(my_model, 
                data_loader,
                pretrain_loss, 
                optimizer = optim.SGD(my_model.parameters(), lr=1e-3, momentum=0.9),
                device = 'cpu',
                gradient_accumulation_steps = 2,
                save_steps = 3,
                epochs = 5)

epochs: 0 begin:================================================================================
1
Step 1, Loss: 8.2541
3
Step 2, Loss: 8.1710
5
Step 3, Loss: 8.1066
Checkpoint saved at steps 3
Step 4, Loss: 6.7037
Finished Training
epochs: 1 begin:================================================================================
1
Step 1, Loss: 8.1069
3
Step 2, Loss: 8.0987
5
Step 3, Loss: 8.0845
Checkpoint saved at steps 3
Step 4, Loss: 6.7730
Finished Training
epochs: 2 begin:================================================================================
1
Step 1, Loss: 8.1232
3
Step 2, Loss: 7.9705
5
Step 3, Loss: 7.9883
Checkpoint saved at steps 3
Step 4, Loss: 6.4137
Finished Training
epochs: 3 begin:================================================================================
1
Step 1, Loss: 7.9293
3
Step 2, Loss: 7.8834
5
Step 3, Loss: 7.7594
Checkpoint saved at steps 3
Step 4, Loss: 6.3352
Finished Training
epochs: 4 begin:====================================================

In [ ]:
'''   
# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')
''' 

In [ ]:
trainer = Trainer(
    model=my_model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

In [62]:
12*(12*768*768 + 13*768) + 30000*768

108094464

In [6]:
anno.drop('Unnamed: 0', axis = 1, inplace = True)
anno.index = anno['sample_name'].values
print(anno.shape)
anno.head()

(1169213, 60)


,cl,sample_name,exp_component_name,donor_label,sex_label,region_label,gene.counts,library_label,platform_label,joint_region_color,...,class_color,ss_cluster_id,ss_cluster_color,ss_cluster_label,tenx_cluster_id,tenx_cluster_color,tenx_cluster_label,cell_type_accession_id,cell_type_accession_color,cell_type_accession_label
10X_cells.ACGCAGCAGACCGGAT-L8TX_180221_01_C11,116,10X_cells.ACGCAGCAGACCGGAT-L8TX_180221_01_C11,ACGCAGCAGACCGGAT-L8TX_180221_01_C11,373822,M,RSP,2703,SM-DPC4K-04,10X,#2B9EAD,...,#F05A28,0,NaN,absent,312,#5E5765,355_V3d,116,#EC2D46,CCN19103010000116
10X_cells.CCATGTCAGCGCTTAT-L8TX_180221_01_C11,128,10X_cells.CCATGTCAGCGCTTAT-L8TX_180221_01_C11,CCATGTCAGCGCTTAT-L8TX_180221_01_C11,373822,M,RSP,2071,SM-DPC4K-04,10X,#2B9EAD,...,#00ADEE,0,NaN,absent,312,#5E5765,355_V3d,364,#413BE2,CCN19103010000364
10X_cells.GCGCAGTAGTGTCTCA-L8TX_180712_01_H05,18,10X_cells.GCGCAGTAGTGTCTCA-L8TX_180712_01_H05,GCGCAGTAGTGTCTCA-L8TX_180712_01_H05,374159,M,TEa-PERI-ECT,3805,SM-DPC4L-01,10X,#76CBE2,...,#F05A28,0,NaN,absent,15,#D97C80,16_Lamp5,18,#D97C80,CCN19103010000018
10X_cells.GTCACGGCACCAACCG-L8TX_180712_01_H05,13,10X_cells.GTCACGGCACCAACCG-L8TX_180712_01_H05,GTCACGGCACCAACCG-L8TX_180712_01_H05,374159,M,TEa-PERI-ECT,4807,SM-DPC4L-01,10X,#76CBE2,...,#F05A28,0,NaN,absent,15,#D97C80,16_Lamp5,15,#A6666F,CCN19103010000015
10X_cells.ATTGGACTCGATCCCT-L8TX_180712_01_A06,18,10X_cells.ATTGGACTCGATCCCT-L8TX_180712_01_A06,ATTGGACTCGATCCCT-L8TX_180712_01_A06,374160,M,TEa-PERI-ECT,3511,SM-DPC4L-02,10X,#76CBE2,...,#F05A28,0,NaN,absent,15,#D97C80,16_Lamp5,18,#D97C80,CCN19103010000018


In [7]:
#useful_col = ['cl','sample_name', 'donor_label', 'sex_label', 'region_label', 'library_label', 'platform_label',
#              ]

#anno.columns

In [8]:
#count = f['data']['counts'][:,:10]

In [9]:
def cell_process(gene_exp, gene_ids, cell_id, meta_info):
    # process cell info into a cell_dic
    # gene_exp: np.array, dtype = int, gene_counts
    # gene_ids: np.array, dtype = object,  gene_id 'ENSMUSG00000051951'
    # cell_id: int, the id of this cell_dic
    # meta_info: a dic, must record the ori dataset and ori cell idx

    cell_dic = {}
    
    cell_dic['cell_id'] = cell_id
    pos_id = (gene_exp > 0)
    cell_dic['pos_gene_ids'] = gene_ids[pos_id]
    cell_dic['pos_gene_exp'] = gene_exp[pos_id] 

    cell_dic['neg_gene_ids'] = gene_ids[~pos_id]

    cell_dic['pos_gene_num'] = cell_dic['pos_gene_ids'].shape[0]
    cell_dic['umi_counts'] = gene_exp.sum()
    
    cell_dic['meta_info'] = meta_info 
    return cell_dic 

def batch_process(gene_exp, gene_ids, cell_id, meta_info):
    # meta info, a dict
    N = gene_exp.shape[0]

    res = []
    for i in tqdm(range(N)):
        cell_dic = cell_process(gene_exp[i], gene_ids, cell_id[i], dict(meta_info.iloc[i,:]))
        res.append(cell_dic)
    return res


In [10]:
data_dir = 'allen_2021_data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
batch_size = 100
gene = f['data']['gene'][:].astype('U')
sample = f['data']['samples'][:].astype('U')

for i in tqdm(range(int(sample.shape[0] // batch_size) + 1)):
    begin = batch_size*i 
    end = min(batch_size*(i+1), sample.shape[0])

    gene_exp = f['data']['counts'][:,begin:end].T

    sample_ids = f['data']['samples'][:].astype('U')
    meta_info = anno.loc[sample_ids]

    gene_ids = gene

    cell_id = [i for i in range(begin, end)]

    res = batch_process(gene_exp, gene, cell_id, meta_info)

    df = pd.DataFrame(res)
    df.to_parquet(os.path.join(data_dir, f'ouput_{i}.parquet'), engine = 'pyarrow')
    
    if i == 1:
        break

  0%|          | 1/11693 [00:08<25:59:29,  8.00s/it]


In [31]:
gene = f['data']['gene'][:].astype('U')
sample = f['data']['samples'][:].astype('U')

numpy.ndarray

In [53]:
sample_ids = f['data']['samples'][:].astype('U')[:10]
sample_ids

array(['10X_cells.AAACCTGAGAAACGCC-L8TX_180221_01_F09',
       '10X_cells.AAACCTGAGAAGGTTT-L8TX_180221_01_F09',
       '10X_cells.AAACCTGAGAGTAATC-L8TX_180221_01_F09',
       '10X_cells.AAACCTGAGCCTTGAT-L8TX_180221_01_F09',
       '10X_cells.AAACCTGAGTGGTAAT-L8TX_180221_01_F09',
       '10X_cells.AAACCTGCAAGCTGTT-L8TX_180221_01_F09',
       '10X_cells.AAACCTGCACAGGAGT-L8TX_180221_01_F09',
       '10X_cells.AAACCTGCACGTCTCT-L8TX_180221_01_F09',
       '10X_cells.AAACCTGCACTTAACG-L8TX_180221_01_F09',
       '10X_cells.AAACCTGGTTGTCGCG-L8TX_180221_01_F09'], dtype='<U46')

In [64]:
dict(anno.iloc[0,:])

{'cl': 116,
 'sample_name': '10X_cells.ACGCAGCAGACCGGAT-L8TX_180221_01_C11',
 'exp_component_name': 'ACGCAGCAGACCGGAT-L8TX_180221_01_C11',
 'donor_label': 373822,
 'sex_label': 'M',
 'region_label': 'RSP',
 'gene.counts': 2703,
 'library_label': 'SM-DPC4K-04',
 'platform_label': '10X',
 'joint_region_color': '#2B9EAD',
 'joint_region_id': 13,
 'joint_region_label': 'RSP',
 'region_id': 20,
 'region_color': '#2B9EAD',
 'sex_color': '#ADC4C3',
 'sex_id': 2,
 'platform_color': '#9F9FED',
 'platform_id': 1,
 'tasic18_cluster_id': 0,
 'tasic18_cluster_label': 'absent',
 'tasic18_cluster_color': '#D9D9D9',
 'tasic18_subclass_id': 0,
 'tasic18_subclass_label': 'absent',
 'tasic18_subclass_color': '#D9D9D9',
 'tasic18_class_id': 0,
 'tasic18_class_label': 'absent',
 'tasic18_class_color': '#D9D9D9',
 'Miniatlas_cluster_id': 0,
 'Miniatlas_cluster_label': 'absent',
 'Miniatlas_cluster_color': '#D9D9D9',
 'genes_label': 2703,
 'genes_norm': 0.278851350333779,
 'genes_color': '#8B8BFF',
 'cluster

In [62]:
anno.loc[sample_ids]

,cl,sample_name,exp_component_name,donor_label,sex_label,region_label,gene.counts,library_label,platform_label,joint_region_color,...,class_color,ss_cluster_id,ss_cluster_color,ss_cluster_label,tenx_cluster_id,tenx_cluster_color,tenx_cluster_label,cell_type_accession_id,cell_type_accession_color,cell_type_accession_label
10X_cells.AAACCTGAGAAACGCC-L8TX_180221_01_F09,179,10X_cells.AAACCTGAGAAACGCC-L8TX_180221_01_F09,AAACCTGAGAAACGCC-L8TX_180221_01_F09,371230,M,ACA,5285,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,139,#02F970,183_L2/3 IT CTX_1,168,#02F970,CCN19103010000168
10X_cells.AAACCTGAGAAGGTTT-L8TX_180221_01_F09,179,10X_cells.AAACCTGAGAAGGTTT-L8TX_180221_01_F09,AAACCTGAGAAGGTTT-L8TX_180221_01_F09,371230,M,ACA,6304,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,139,#02F970,183_L2/3 IT CTX_1,168,#02F970,CCN19103010000168
10X_cells.AAACCTGAGAGTAATC-L8TX_180221_01_F09,196,10X_cells.AAACCTGAGAGTAATC-L8TX_180221_01_F09,AAACCTGAGAGTAATC-L8TX_180221_01_F09,371230,M,ACA,5285,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,150,#09CCC6,197_L4/5 IT CTX,189,#09CCC6,CCN19103010000189
10X_cells.AAACCTGAGCCTTGAT-L8TX_180221_01_F09,340,10X_cells.AAACCTGAGCCTTGAT-L8TX_180221_01_F09,AAACCTGAGCCTTGAT-L8TX_180221_01_F09,371230,M,ACA,3503,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,277,#2E8CCB,327_L6 CT CTX,290,#3E98A5,CCN19103010000290
10X_cells.AAACCTGAGTGGTAAT-L8TX_180221_01_F09,312,10X_cells.AAACCTGAGTGGTAAT-L8TX_180221_01_F09,AAACCTGAGTGGTAAT-L8TX_180221_01_F09,371230,M,ACA,3742,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,259,#45BE72,304_L5 NP CTX,265,#45BE72,CCN19103010000265
10X_cells.AAACCTGCAAGCTGTT-L8TX_180221_01_F09,182,10X_cells.AAACCTGCAAGCTGTT-L8TX_180221_01_F09,AAACCTGCAAGCTGTT-L8TX_180221_01_F09,371230,M,ACA,4043,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,104,#B1EC30,186_L2 IT RSP-ACA,134,#B1EC30,CCN19103010000134
10X_cells.AAACCTGCACAGGAGT-L8TX_180221_01_F09,196,10X_cells.AAACCTGCACAGGAGT-L8TX_180221_01_F09,AAACCTGCACAGGAGT-L8TX_180221_01_F09,371230,M,ACA,5586,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,150,#09CCC6,197_L4/5 IT CTX,189,#09CCC6,CCN19103010000189
10X_cells.AAACCTGCACGTCTCT-L8TX_180221_01_F09,6,10X_cells.AAACCTGCACGTCTCT-L8TX_180221_01_F09,AAACCTGCACGTCTCT-L8TX_180221_01_F09,371230,M,ACA,4408,SM-DPC4J-01,10X,#FFDE17,...,#F05A28,0,NaN,absent,9,#935F50,3_Lamp5 Lhx6,6,#9E7153,CCN19103010000006
10X_cells.AAACCTGCACTTAACG-L8TX_180221_01_F09,179,10X_cells.AAACCTGCACTTAACG-L8TX_180221_01_F09,AAACCTGCACTTAACG-L8TX_180221_01_F09,371230,M,ACA,5571,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,139,#02F970,183_L2/3 IT CTX_1,168,#02F970,CCN19103010000168
10X_cells.AAACCTGGTTGTCGCG-L8TX_180221_01_F09,180,10X_cells.AAACCTGGTTGTCGCG-L8TX_180221_01_F09,AAACCTGGTTGTCGCG-L8TX_180221_01_F09,371230,M,ACA,5340,SM-DPC4J-01,10X,#FFDE17,...,#00ADEE,0,NaN,absent,139,#02F970,183_L2/3 IT CTX_1,169,#1AD475,CCN19103010000169


In [38]:
sample

array(['10X_cells.AAACCTGAGAAACGCC-L8TX_180221_01_F09',
       '10X_cells.AAACCTGAGAAGGTTT-L8TX_180221_01_F09',
       '10X_cells.AAACCTGAGAGTAATC-L8TX_180221_01_F09', ...,
       '10X_cells.TTTGTCAGTGAGGCTA-L8TX_200611_02_A05',
       '10X_cells.TTTGTCATCAGGCGAA-L8TX_200611_02_A05',
       '10X_cells.TTTGTCATCGTCTGAA-L8TX_200611_02_A05'], dtype='<U46')